In [1]:
from keras import backend as K
from keras.optimizers import Adadelta
from keras.callbacks import EarlyStopping, ModelCheckpoint
from Image_Generator import TextImageGenerator
from Model import get_Model
from parameter import *
K.set_learning_phase(0)

Using TensorFlow backend.


In [2]:
import tensorflow as tf
tf.__version__
tf.debugging.set_log_device_placement(True)
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [3]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8855516450381707592
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6620742943
locality {
  bus_id: 1
  links {
  }
}
incarnation: 14313668320821753477
physical_device_desc: "device: 0, name: GeForce RTX 2070, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


In [ ]:
model = get_Model(training=True)

try:
    #model.load_weights('Final_weight.hdf5')
    #model.load_weights('LSTM+BN5--25--0.402.hdf5')
    print("...Previous weight data...")
except:
    print("...New weight data...")
    pass

train_file_path = './DB/train/'
tiger_train = TextImageGenerator(train_file_path, img_w, img_h, batch_size, downsample_factor)
tiger_train.build_data()

valid_file_path = './DB/test/'
tiger_val = TextImageGenerator(valid_file_path, img_w, img_h, val_batch_size, downsample_factor)
tiger_val.build_data()



In [ ]:

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)



In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

In [ ]:

ada = Adadelta()

early_stop = EarlyStopping(monitor='loss', min_delta=0.001, patience=4, mode='min', verbose=1)
checkpoint = ModelCheckpoint(filepath='LSTM+BN5--{epoch:02d}--{val_loss:.3f}.hdf5', monitor='loss', verbose=1, mode='min', period=1)
# the loss calc occurs elsewhere, so use a dummy lambda func for the loss
model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=ada)

# captures output of softmax so we can decode the output during visualization
model.fit_generator(generator=tiger_train.next_batch(),
                    steps_per_epoch=int(tiger_train.n / batch_size),
                    epochs=50,
                    callbacks=[checkpoint],
                    validation_data=tiger_val.next_batch(),
                    validation_steps=int(tiger_val.n / val_batch_size))

In [5]:
!python Prediction.py -w "lasteset_weight_after_rename.hdf5" -t "./DB/test2/"

...Previous weight data...
pred_texts: A03gj8400X
Predicted: 서울 03허8400  /  True: 서울 03허8400
pred_texts: A11en0752X
Predicted: 서울 11두0752  /  True: 서울 11두0752
pred_texts: A14ah8695X

Using TensorFlow backend.





2020-08-10 11:07:25.645250: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2
2020-08-10 11:07:25.647891: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library nvcuda.dll
2020-08-10 11:07:25.671512: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1640] Found device 0 with properties: 
name: GeForce RTX 2070 major: 7 minor: 5 memoryClockRate(GHz): 1.62
pciBusID: 0000:01:00.0
2020-08-10 11:07:25.671763: I tensorflow/stream_executor/platform/default/dlopen_checker_stub.cc:25] GPU libraries are statically linked, skip dlopen check.
2020-08-10 11:07:25.671992: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1763] Adding visible gpu devices: 0
2020-08-10 11:07:26.065826: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1181] Device interconnect StreamExecutor with strength 1 edge matrix:
2020-08-10 11:07:26.066019: I 


Predicted: 서울 14모8695  /  True: 서울 14모8695
pred_texts: A35tn1293X
Predicted: 서울 35수1293  /  True: 서울 35수1293
pred_texts: A51ah2548X
Predicted: 서울 51모2548  /  True: 서울 51모2548
pred_texts: A52tn0750X
Predicted: 서울 52수0750  /  True: 서울 52수0750
pred_texts: A66en1240X
Predicted: 서울 66두1240  /  True: 서울 66두1240
pred_texts: A78gj4117X
Predicted: 서울 78허4117  /  True: 서울 78허4117
pred_texts: B01wh0226X
Predicted: 경기 01조0226  /  True: 경기 01조0226
pred_texts: B21qh8019X
Predicted: 경기 21보8019  /  True: 경기 21보8019
pred_texts: B38aj7891X
Predicted: 경기 38머7891  /  True: 경기 38머7891
pred_texts: B43aj4072X
Predicted: 경기 43머4072  /  True: 경기 43머4072
pred_texts: B48fh0941X
Predicted: 경기 48로0941  /  True: 경기 48로0941
pred_texts: B50fh1923X
Predicted: 경기 50로1923  /  True: 경기 50로1923
pred_texts: B52wh3012X
Predicted: 경기 52조3012  /  True: 경기 52조3012
pred_texts: B64qh5302X
Predicted: 경기 64보5302  /  True: 경기 64보5302
pred_texts: C17ak3400X
Predicted: 인천 17마3400  /  True: 인천 17마3400
pred_texts: C18qj7981X
Predicted




Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.cast` instead.


Predicted: 전남 34로9121  /  True: 전남 34로9121
pred_texts: M53dn4704X
Predicted: 전남 53우4704  /  True: 전남 53우4704
pred_texts: M86dn5844X
Predicted: 전남 86우5844  /  True: 전남 86우5844
pred_texts: N02fj3184X
Predicted: 광주 02러3184  /  True: 광주 02러3184
pred_texts: N24th7463X
Predicted: 광주 24소7463  /  True: 광주 24소7463
pred_texts: N36eh1467X
Predicted: 광주 36도1467  /  True: 광주 36도1467
pred_texts: N67fj6925X
Predicted: 광주 67러6925  /  True: 광주 67러6925
pred_texts: N78eh2754X
Predicted: 광주 78도2754  /  True: 광주 78도2754
pred_texts: N82th0912X
Predicted: 광주 82소0912  /  True: 광주 82소0912
pred_texts: O02fk2246X
Predicted: 전북 02라2246  /  True: 전북 02라2246
pred_texts: O63tj2535X
Predicted: 전북 63서2535  /  True: 전북 63서2535
pred_texts: O71tj0089X
Predicted: 전북 71서0089  /  True: 전북 71서0089
pred_texts: O77ej4209X
Predicted: 전북 77더4209  /  True: 전북 77더4209
pred_texts: O82fk9548X
Predicted: 전북 82라9548  /  True: 전북 82라9548
pred_texts: O99ej6262X
Predicted: 전북 99더6262  /  True: 전북 99더6262
pred_texts: P03ek8078X
Predicted:

In [ ]:
!python Prediction.py -w "LSTM+BN5--29--0.250.hdf5" -t "./DB/test/"

In [ ]:
!python Prediction.py -w "LSTM+BN5--05--3.136.hdf5" -t "./DB/test2/"